# 说明：

使用的数据是MNIST手写数字数据集；用tf.data来自己创建TF能识别的Tensor数据。

In [2]:
import tensorflow as tf

In [3]:
# 加载数据集:
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

In [4]:
# 原始的数据都是ndarray，虽然可以直接用！但预先转为tensor数据类型会更好更快（因为张量快）：以后也要这么写！！
type(train_image)

numpy.ndarray

# 数据转换：

In [5]:
# 训练集转化：数据 + 标签
ds_train_image = tf.data.Dataset.from_tensor_slices( train_image )
ds_train_label = tf.data.Dataset.from_tensor_slices( train_label )

In [6]:
# 把数据和标签“一一对应”联系在一起（对应组件合并为一个组件）：之后的变换，要变一起变！
ds_train = tf.data.Dataset.zip( (ds_train_image, ds_train_label) )  # 注：要求元组的形式（同numpy用法）！

In [7]:
ds_train  # 后者是()，因为label就是一个数！

<ZipDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>

In [10]:
# 随机打散：对合并后组件的处理
# 打散：60000个元素中取10000个进行打乱，无限重复打乱过程 
# 分批次：每个批次64个
ds_train = ds_train.shuffle(10000).repeat().batch(64) # 每一次输入64张，一个epoch下又会输入60000/64 = 938次！

In [11]:
train_image.shape

(60000, 28, 28)

# 模型训练：

In [12]:
model = tf.keras.Sequential()

In [13]:
model.add( tf.keras.layers.Flatten(input_shape = (28,28)) )
model.add( tf.keras.layers.Dense(128, activation = 'relu') )
model.add( tf.keras.layers.Dense(10, activation = 'softmax') )

In [14]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

In [17]:
# 因为有了repeat()，导致可迭代对象(每个epoch)是无限循环的，需要人为设置每个epoch中到底有多少个batch！
steps_per_epoch = train_image.shape[0] // 64  # 必须是整数

In [18]:
# 直接对ds_train进行训练：既包含数据，又包含对应的标签
# 原来是每个epoch一次性放入60000张图；现在是每个epoch中又分937次放入，每次放入64张！—— 实质没变
model.fit( ds_train, epochs = 5, steps_per_epoch = steps_per_epoch )

Train for 937 steps
Epoch 1/5
937/937 [==============================] - 6s 6ms/step - loss: 3.1579 - acc: 0.8511
Epoch 2/5
937/937 [==============================] - 4s 4ms/step - loss: 0.4434 - acc: 0.9043
Epoch 3/5
937/937 [==============================] - 4s 4ms/step - loss: 0.2916 - acc: 0.9287
Epoch 4/5
937/937 [==============================] - 4s 4ms/step - loss: 0.2421 - acc: 0.9402
Epoch 5/5
937/937 [==============================] - 4s 4ms/step - loss: 0.2055 - acc: 0.9482


# 把测试数据也加入：validation_data —— 标准写法！

说明：操作和train是完全一样的。

改进：其实**tf.data.Dataset.zip都可以省略的**。直接在tf.data.Dataset.from_tensor_slices( (数据,标签) ) —— 一起放入就好。

---
对测试数据没必要shuffle，但依旧需要batch（因为一次性放入10000张还是太多！）

In [20]:
# 数据转换：改进 —— 直接元组形式放入，直接拼凑好！

# 训练集：数据 + 标签
ds_train = tf.data.Dataset.from_tensor_slices( (train_image, train_label) )
# 测试集：数据 + 标签
ds_test = tf.data.Dataset.from_tensor_slices( (test_image, test_label) )

In [21]:
ds_train, ds_test

(<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>,
 <TensorSliceDataset shapes: ((28, 28), ()), types: (tf.uint8, tf.uint8)>)

In [22]:
# 乱序：训练、测试数据都要进行！
ds_train = ds_train.shuffle(60000).repeat().batch(64)
ds_test = ds_test.repeat().batch(64)

# 同样，因为都加了repeat，都需要人为设置每个epoch中的batch的数量：为了让repeat停下来！
train_per_epochs = train_image.shape[0] // 64
test_per_epochs = test_image.shape[0] // 64

In [23]:
model = tf.keras.Sequential()

model.add( tf.keras.layers.Flatten(input_shape = (28,28)) )
model.add( tf.keras.layers.Dense(128, activation = 'relu') )
model.add( tf.keras.layers.Dense(10, activation = 'softmax') )

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 模型训练：训练集、测试集都还是每个epoch的结果（只不过输送数据还是按batch来）
# 打印结果还是按一个epoch来的（不是按batch来的，这是肯定的）！
model.fit( 
    ds_train, 
    epochs = 5, 
    steps_per_epoch = train_per_epochs,  # 训练数据部分
    validation_data = ds_test,
    validation_steps = test_per_epochs   # 测试数据部分
)

Train for 937 steps, validate for 156 steps
Epoch 1/5
937/937 [==============================] - 5s 5ms/step - loss: 3.5022 - acc: 0.8747 - val_loss: 0.7747 - val_acc: 0.8880
Epoch 2/5
937/937 [==============================] - 4s 5ms/step - loss: 0.4756 - acc: 0.9107 - val_loss: 0.4535 - val_acc: 0.9169
Epoch 3/5
937/937 [==============================] - 5s 5ms/step - loss: 0.2947 - acc: 0.9315 - val_loss: 0.3982 - val_acc: 0.9229
Epoch 4/5
937/937 [==============================] - 5s 5ms/step - loss: 0.2398 - acc: 0.9403 - val_loss: 0.3374 - val_acc: 0.9254
Epoch 5/5
937/937 [==============================] - 5s 5ms/step - loss: 0.2148 - acc: 0.9468 - val_loss: 0.2942 - val_acc: 0.9370
